In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("Data/spam.tsv", sep="\t")
df.head(10)

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2
5,spam,FreeMsg Hey there darling it's been 3 week's n...,147,8
6,ham,Even my brother is not like to speak with me. ...,77,2
7,ham,As per your request 'Melle Melle (Oru Minnamin...,160,6
8,spam,WINNER!! As a valued network customer you have...,157,6
9,spam,Had your mobile 11 months or more? U R entitle...,154,2


### Check for Null Values

In [3]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [4]:
df.shape

(5572, 4)

In [5]:
df["label"].value_counts()

label
ham     4825
spam     747
Name: count, dtype: int64

Note: Data is not balanced.

To balance, the ham messages will be undersampled.

## Dataset Balancing

In [6]:
# filter ham
ham = df[df["label"] == "ham"]

ham.shape

(4825, 4)

In [7]:
spam = df[df["label"]=="spam"]
spam.shape

(747, 4)

In [8]:
ham = ham.sample(spam.shape[0])
ham.shape

(747, 4)

Concatenate ham and spam messages,row-wise

In [9]:
data = pd.concat([ham, spam], axis=0,ignore_index=True)
data.shape

(1494, 4)

In [11]:
data.sample(10)

,label,message,length,punct
1037,spam,Well done ENGLAND! Get the official poly ringt...,155,6
996,spam,SMS AUCTION You have won a Nokia 7250i. This i...,162,7
758,spam,Congrats! 1 year special cinema pass for 2 is ...,161,12
1206,spam,Am new 2 club & dont fink we met yet Will B gr...,160,2
1302,spam,URGENT! Your Mobile number has been awarded a ...,141,6
888,spam,Your free ringtone is waiting to be collected....,158,8
1113,spam,URGENT! We are trying to contact U. Todays dra...,157,5
1287,spam,Please call our customer service representativ...,147,2
459,ham,Then wat r u doing now? Busy wif work?,38,2
1072,spam,URGENT! Your mobile number *************** WON...,166,22


### Exploratory Data Analysis